#  LangChain의 RAG 콤포넌트 - 벡터 저장소(Vector Store) 

### **학습 목표:**  임베딩 모델과 벡터 데이터베이스를 효과적으로 연동할 수 있다

### **사전 준비:**

- 필요 패키지: `langchain-chroma`, `faiss-cpu`, `langchain-pinecone`
- 환경변수: OPENAI_API_KEY, PINECONE_API_KEY 설정 필요

### **실습 자료**: 해당 없음

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

# 벡터 저장소 (Vector Store)

- 개념:
    - 벡터화된 데이터를 효율적으로 저장하고 검색하기 위한 특수 데이터베이스 시스템
    - 텍스트나 이미지 등의 비정형 데이터를 고차원 벡터 공간에 매핑하여 저장
    - 유사도 기반 검색을 통해 의미적으로 가까운 데이터를 빠르게 검색 가능 

- LangChain의 벡터 저장소 종류:
    - **Chroma**: 경량화된 임베딩 데이터베이스로 로컬 개발에 적합
    - **FAISS**: Facebook AI가 개발한 고성능 유사도 검색 라이브러리
    - **Pinecone**: 완전 관리형 벡터 데이터베이스 서비스
    - Milvus: 분산 벡터 데이터베이스로 대규모 데이터 처리에 적합
    - PostgreSQL: pgvector 확장을 통해 벡터 저장 및 검색 기능을 제공

- 주요 기능:
    - 벡터 색인화: 효율적인 검색을 위한 데이터 구조화를 수행
    - 근접 이웃 검색: 주어진 쿼리와 가장 유사한 벡터들을 검색 
    - 메타데이터 관리: 벡터와 관련된 부가 정보를 함께 저장하고 검색

- 사용 사례:
    - 시맨틱 문서 검색: 문서의 의미를 이해하여 검색
    - 추천 시스템: 유사한 아이템을 추천
    - 중복 데이터 감지: 유사한 콘텐츠를 검색 
    - 질의응답 시스템: 관련 문서에서 답변을 생성하는데 필요한 근거를 검색 

- 벡터 저장소 선택 가이드

    | 벡터 저장소 | 장점 | 단점 | 추천 사용 사례 |
    |------------|------|------|---------------|
    | **Chroma** | 설정 간단, 로컬 개발 용이 | 대규모 데이터 처리 제한적 | 프로토타입, 소규모 프로젝트 |
    | **FAISS** | 빠른 검색 속도, 대용량 처리 | 별도 저장소 없음 (메모리/파일만) | 고성능 검색, 대규모 데이터 |
    | **Pinecone** | 완전 관리형, 확장성 우수 | 유료 서비스, API 키 필요 | 프로덕션 환경, 클라우드 배포 |

### 1. **Chroma**

- 사용자 편의성이 우수한 오픈소스 벡터 저장소
- `langchain-chroma` 패키지 설치

`(1) 벡터 저장소 초기화`

In [ ]:
# 벡터 저장소에 문서를 저장할 때 적용할 임베딩 모델
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

In [ ]:
# from langchain_ollama import OllamaEmbeddings 

# embeddings_model = OllamaEmbeddings(model="dengcao/Qwen3-Embedding-0.6B:Q8_0")

In [ ]:
# 벡터 저장소 생성
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_sample_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)

In [ ]:
# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [ ]:
from langchain_core.documents import Document

# 문서 데이터 - (텍스트, 소스)
documents = [
    ("인공지능은 컴퓨터 과학의 한 분야입니다.", "AI 개론"),
    ("머신러닝은 인공지능의 하위 분야입니다.", "AI 개론"),
    ("딥러닝은 머신러닝의 한 종류입니다.", "딥러닝 입문"),
    ("자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.", "AI 개론"),
    ("컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.", "딥러닝 입문")
]

# Document 객체 생성
doc_objects = []
for content, source in documents:
    doc = Document(
        page_content=content,
        metadata={"source": source},
    )
    doc_objects.append(doc)


# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
added_doc_ids = chroma_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

In [ ]:
# 저장된 문서 검색
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

In [ ]:
# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

- 문서 수정: `vector_store.update_document(document_id, document)`

In [ ]:
# 업데이트할 문서 생성
updated_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.",
    metadata={"source": "AI 개론"},
)

updated_document_2 = Document(
    page_content="머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.",
    metadata={"source": "AI 개론"},
)

updated_document_3 = Document(
    page_content="딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.",
    metadata={"source": "딥러닝 입문"},
)


# 단일 문서 업데이트
chroma_db.update_document(document_id="DOC_1", document=updated_document_1)

# 여러 문서 한 번에 업데이트
chroma_db.update_documents(
    ids=["DOC_2", "DOC_3"],
    documents=[updated_document_2, updated_document_3]
)

print("문서 업데이트 완료")

In [ ]:
# 저장된 문서 검색 예시
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

- 문서 삭제: `vector_store.delete(ids)`

In [ ]:
# 문서 id를 지정하여 삭제
chroma_db.delete(ids=["DOC_5"])

In [ ]:
# 컬렉션 확인
chroma_db.get()

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    -  k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [ ]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = chroma_db.similarity_search(
    query,
    k=2,
    filter={"source": "딥러닝 입문"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

**유사도 점수 해석**

- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미 (거리 기준으로 점수가 산정되기 때문)


- 검색 방법 비교

    | 메서드 | 반환값 | 점수 범위 | 점수 해석 |
    |--------|--------|-----------|-----------|
    | `similarity_search` | 문서 리스트만 | - | 점수 없음 |
    | `similarity_search_with_score` | (문서, 거리점수) | 0 ~ ∞ | **낮을수록** 유사 |
    | `similarity_search_with_relevance_scores` | (문서, 관련성점수) | 0 ~ 1 | **높을수록** 유사 |



In [ ]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [ ]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

`(4) 벡터 저장소 로드`  

In [ ]:
chroma_db2 = Chroma(
    collection_name="ai_sample_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)

In [ ]:
# 미리 임베딩된 쿼리 벡터를 사용하여 검색
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "딥러닝 입문"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

### **2 FAISS(Facebook AI Similarity Search)**

- 효율적인 벡터 유사도 검색 및 클러스터링을 위한 오픈소스 벡터 저장소 
- `faiss-cpu` 패키지 설치

`(1) 벡터 저장소 초기화`

In [ ]:
embeddings_model

In [ ]:
len(embeddings_model.embed_query("hello world"))

In [ ]:
# 벡터 저장소 생성
import faiss 
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# FAISS 인덱스 초기화 (유클리드 거리 사용)
faiss_index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
print("FAISS 인덱스 초기화 완료")

In [ ]:
# FAISS 벡터 저장소의 벡터 차원 수 (임베딩 차원 수)
faiss_index.d

In [ ]:
# FAISS 벡터 저장소 생성
faiss_db = FAISS(
    embedding_function=embeddings_model,
    index=faiss_index,           # 벡터 검색을 위한 데이터 구조를 정의
    docstore=InMemoryDocstore(), # 문서 저장소 객체를 지정 - 문서의 원본 내용과 메타데이터를 보관
    index_to_docstore_id={},     # 인덱스와 문서 간의 연결을 관리 (매핑 딕셔너리)
)
# 저장된 문서의 갯수 확인
faiss_db.index.ntotal

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [ ]:
from langchain_core.documents import Document

# 문서 데이터 - (텍스트, 소스)
documents = [
    ("인공지능은 컴퓨터 과학의 한 분야입니다.", "AI 개론"),
    ("머신러닝은 인공지능의 하위 분야입니다.", "AI 개론"),
    ("딥러닝은 머신러닝의 한 종류입니다.", "딥러닝 입문"),
    ("자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.", "AI 개론"),
    ("컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.", "딥러닝 입문")
]

# Document 객체 생성
doc_objects = []
for content, source in documents:
    doc = Document(
        page_content=content,
        metadata={"source": source},
    )
    doc_objects.append(doc)


# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
added_doc_ids = faiss_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

In [ ]:
# 저장된 문서의 갯수 확인
faiss_db.index.ntotal

In [ ]:
# 저장된 인덱스 확인
faiss_db.index_to_docstore_id

In [ ]:
# 저장된 문서 검색
faiss_db.docstore.search('DOC_1')

- 문서 삭제: `vector_store.delete(ids)`

In [ ]:
# 문서 id를 지정하여 삭제
faiss_db.delete(ids=["DOC_5"])

In [ ]:
# 컬렉션 확인
faiss_db.index.ntotal

In [ ]:
# 저장된 인덱스 확인
faiss_db.index_to_docstore_id

In [ ]:
# 저장된 문서 객체를 확인
faiss_db.docstore._dict

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    - k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [ ]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = faiss_db.similarity_search(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미

In [ ]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [ ]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

`(4) 로컬에 저장 및 로드`  

In [ ]:
# 로컬에 저장
faiss_db.save_local("faiss_ai_sample_index")

In [ ]:
# 로컬에 저장된 FAISS 벡터 저장소 불러오기
faiss_db2 = FAISS.load_local(
    "faiss_ai_sample_index", embeddings_model, allow_dangerous_deserialization=True
)

In [ ]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "딥러닝 입문"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

### 3. **Pinecone**

- 상용 클라우드 기반의 벡터 데이터베이스 서비스 (SaaS)
- 실시간 고성능 벡터 검색 제공
- 회원가입 및 API 인증키 발급 (.env 파일에 `PINECONE_API_KEY` 등록)
- `langchain-pinecone pinecone-notebooks` 패키지 설치

`(1) 환경 설정`

In [ ]:
# PINECONE_API_KEY 환경 변수 설정 후에 실행
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
# 인증 설정
import os
from pinecone import Pinecone, ServerlessSpec 
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
if not pinecone_api_key:
    raise ValueError("PINECONE_API_KEY가 .env 파일에 설정되지 않았습니다.")
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
pc.list_indexes()

In [ ]:
# 기존 인덱스 리스트 확인
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
print(f"기존 인덱스: {existing_indexes}")

`(2) 벡터 저장소 초기화`

In [ ]:
import time 

# 인덱스 이름 설정
index_name = "ai-sample-index"

# 인덱스가 없으면 생성
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="euclidean",  # 유사도 측정 방법 - euclidean, cosine, dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

# 인덱스 이름을 사용하여 인덱스 객체 생성
index = pc.Index(index_name)

In [ ]:
# 인덱스 정보 확인
index_name = "ai-sample-index"

index_info = pc.describe_index(index_name)
index_info

In [ ]:
from langchain_pinecone import PineconeVectorStore  

# PINECONE 벡터 저장소 생성
pinecone_db = PineconeVectorStore(index=index, embedding=embeddings_model)

# 벡터 저장소 객체 확인 
pinecone_db

In [ ]:
# 저장된 문서의 갯수 확인
pinecone_db._index.describe_index_stats()

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [ ]:
from langchain_core.documents import Document

# 문서 데이터 - (텍스트, 소스)
documents = [
    ("인공지능은 컴퓨터 과학의 한 분야입니다.", "AI 개론"),
    ("머신러닝은 인공지능의 하위 분야입니다.", "AI 개론"),
    ("딥러닝은 머신러닝의 한 종류입니다.", "딥러닝 입문"),
    ("자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.", "AI 개론"),
    ("컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.", "딥러닝 입문")
]

# Document 객체 생성
doc_objects = []
for content, source in documents:
    doc = Document(
        page_content=content,
        metadata={"source": source},
    )
    doc_objects.append(doc)


# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
added_doc_ids = pinecone_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

In [ ]:
# 저장된 문서의 갯수 확인 - 동기화에 시간이 걸릴 수 있음
pinecone_db._index.describe_index_stats()

In [ ]:
# 저장된 문서 검색
query = "인공지능과 머신러닝의 관계는?"
results = pinecone_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

- 문서 삭제: `vector_store.delete(ids)`

In [ ]:
# 문서 id를 지정하여 삭제
pinecone_db.delete(ids=["DOC_5"])

In [ ]:
# 컬렉션 확인
pinecone_db._index.describe_index_stats()

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    -  k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [ ]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = pinecone_db.similarity_search(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미 (거리 기준으로 점수가 산정되기 때문)

In [ ]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = pinecone_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [ ]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = pinecone_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

`(4) 벡터 저장소 로드`  

In [ ]:
# 저장된 인덱스 확인해서 초기화 
index_name = "ai-sample-index"
index = pc.Index(index_name)
pinecone_db2 = PineconeVectorStore(index=index, embedding=embeddings_model)

# 저장된 문서 정보를 확인
pinecone_db2._index.describe_index_stats()

In [ ]:
# 미리 임베딩된 쿼리 벡터를 사용하여 검색
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = pinecone_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

# [실습 프로젝트]

### 실습 목표

1. 아래 샘플 문서들을 벡터 저장소에 저장하는 코드를 작성합니다. 
   - 적절한 벡터 저장소 선택 
   - 임베딩 모델 설정
   - 문서 구조 설계 (metadata 정의)

2. 벡터 저장소를 사용하여 다음 기능을 구현합니다. 
   - 새로운 문서 추가
   - 문서 삭제
   - 문서 검색: 유사도 점수 계산, 메타데이터 기반 필터링 등 

### 실습 단계

- **1단계: 벡터 저장소 초기화**
   - Chroma, FAISS, Pinecone 중 하나를 선택하고 이유를 설명하세요

- **2단계: 문서 저장**
   - 제공된 샘플 문서를 Document 객체로 변환하세요
   - metadata 구조를 설계하세요 (type, author 포함)

- **3단계: 문서 관리**
   - 새로운 문서 1개를 추가하세요
   - 특정 문서 1개를 삭제하세요

- **4단계: 문서 검색 구현**
   - [ ] 기본 유사도 검색
   - [ ] 메타데이터 필터링
   - [ ] 점수 포함 검색

In [ ]:
# 샘플 문서 데이터 
documents = [
    {"content": "인공지능 기술의 발전과 미래", "type": "article", "author": "김철수"},
    {"content": "데이터 분석 입문 가이드", "type": "tutorial", "author": "이영희"},
    {"content": "머신러닝 모델 성능 개선 방법", "type": "research", "author": "박지성"},
    {"content": "블록체인 기술과 금융 혁신", "type": "article", "author": "정민우"},
    {"content": "클라우드 컴퓨팅 아키텍처 설계", "type": "tutorial", "author": "강다은"},
    {"content": "사이버 보안 위협 대응 전략", "type": "research", "author": "홍길동"},
    {"content": "빅데이터 처리 시스템 구축 사례", "type": "article", "author": "송지원"},
    {"content": "웹 개발자를 위한 REST API 가이드", "type": "tutorial", "author": "임성준"},
    {"content": "자연어 처리 알고리즘 비교 연구", "type": "research", "author": "최유진"},
    {"content": "디지털 트랜스포메이션 성공 전략", "type": "article", "author": "백승호"},
    {"content": "파이썬으로 시작하는 데이터 시각화", "type": "tutorial", "author": "유미란"},
    {"content": "강화학습을 활용한 게임 AI 개발", "type": "research", "author": "조현우"},
    {"content": "5G 네트워크 기술 동향", "type": "article", "author": "윤서연"},
    {"content": "도커 컨테이너 실전 가이드", "type": "tutorial", "author": "장민석"},
    {"content": "추천 시스템 최적화 연구", "type": "research", "author": "신영수"},
    {"content": "스마트 시티 구현 기술", "type": "article", "author": "권태영"},
    {"content": "깃허브 활용 협업 가이드", "type": "tutorial", "author": "오지훈"},
    {"content": "컴퓨터 비전 응용 사례 연구", "type": "research", "author": "남궁민"},
    {"content": "양자 컴퓨팅의 현재와 미래", "type": "article", "author": "하은주"},
    {"content": "리액트 네이티브 앱 개발 입문", "type": "tutorial", "author": "문동현"},
    {"content": "음성인식 시스템 성능 평가", "type": "research", "author": "심준호"},
    {"content": "메타버스 플랫폼 개발 동향", "type": "article", "author": "류아린"},
    {"content": "NoSQL 데이터베이스 설계 패턴", "type": "tutorial", "author": "반승현"},
    {"content": "엣지 컴퓨팅 적용 사례 연구", "type": "research", "author": "주민정"},
    {"content": "디지털 헬스케어 기술 혁신", "type": "article", "author": "구본우"},
    {"content": "마이크로서비스 아키텍처 구현", "type": "tutorial", "author": "염지현"},
    {"content": "강화학습 기반 로봇 제어 연구", "type": "research", "author": "탁현우"},
    {"content": "친환경 IT 인프라 구축 방안", "type": "article", "author": "방승미"},
    {"content": "프론트엔드 성능 최적화 기법", "type": "tutorial", "author": "곽준영"},
    {"content": "시계열 데이터 예측 모델 연구", "type": "research", "author": "추민서"}
]

In [ ]:
# Document 객체 생성
from langchain_core.documents import Document

doc_objects = []

for i, doc in enumerate(documents): 
    doc_obj = Document(
        page_content=doc["content"],
        metadata={"type": doc["type"], "author": doc["author"]},
    )
    doc_objects.append(doc_obj)

# uuid 생성
import uuid

doc_ids = [str(uuid.uuid4()) for _ in range(len(doc_objects))]

### 여기에 나머지 코드를 작성하세요 ###

